(C) 2023 Gaudeor Rudmin

<if you edit this code, add your copyright statement here>

MIT LICENSE

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### Enter the headers and names of the of the tables

In [16]:
#####    CUSTOMISATION    ######
just_show = True # if True, only selects (which displays results on the SQL page)
select_no_masers = False ## if True, saves non-masers

#name of the database to create with no masers
target_dbname = 'MyDB.all_surveyed_no_maser_no_duplicates' 


#arcsec angular separation to assume that two objects in the different lists are the same object
# Be sure to use floating point notation, and use a string.
assume_same_object_radius = '6.0' 

## create a key-lookup table
# Column identifiers for all_surveyed
all_surveyed_dbname = 'all_surveyed_no_dupes_cleansed4x'

all_surveyed_cols_ra = 'ra'                 #RA header
all_surveyed_cols_dec = 'dec'               #DEC header
all_surveyed_cols_uid = 'mcp_count'         # a unique ID for each row in all_surveyed


# Column identifiers for masers
masers_dbname = 'MCPMaser2020'

masers_cols_ra = 'RA_(J2000)'               #RA header
masers_cols_dec = 'Dec_(J2000)'             #DEC header

#####    /CUSTOMISATION/    ######


dowhat = f'''
INTO {target_dbname}'''
if just_show:
    dowhat = ""

selection = "IN"
if select_no_masers:
    selection = "NOT IN"

print(f'''SELECT *
FROM {all_surveyed_dbname} S
WHERE 
    S.[{all_surveyed_cols_uid}] {selection} (
        SELECT S.[{all_surveyed_cols_uid}]
        FROM {all_surveyed_dbname} S, {masers_dbname} M
        WHERE
        -- Small angular distance approximation
        ({assume_same_object_radius}/3600.0) > ( SQRT((POWER( ( ( (S.[{all_surveyed_cols_ra}] * (PI()/180.0)) - (M.[{masers_cols_ra}] * (PI()/180.0)) )*( COS( (S.[{all_surveyed_cols_dec}] * (PI()/180.0)) )) ),2) )+(POWER(( (S.[{all_surveyed_cols_dec}] * (PI()/180.0)) - (M.[{masers_cols_dec}] * (PI()/180.0)) ),2))) ) * (180.0/PI())
    ){dowhat}
''')

SELECT *
FROM all_surveyed_no_dupes_cleansed4x S
WHERE 
    S.[mcp_count] IN (
        SELECT S.[mcp_count]
        FROM all_surveyed_no_dupes_cleansed4x S, MCPMaser2020 M
        WHERE
        -- Small angular distance approximation
        (6.0/3600.0) > ( SQRT((POWER( ( ( (S.[ra] * (PI()/180.0)) - (M.[RA_(J2000)] * (PI()/180.0)) )*( COS( (S.[dec] * (PI()/180.0)) )) ),2) )+(POWER(( (S.[dec] * (PI()/180.0)) - (M.[Dec_(J2000)] * (PI()/180.0)) ),2))) ) * (180.0/PI())
    )

